In [8]:
import pandas as pd
import numpy as np
import os
import datetime

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout
from tensorflow.keras.utils import to_categorical

%load_ext tensorboard

import matplotlib.pyplot as plt
from skimage import color, exposure

from sklearn.metrics import accuracy_score

from hyperopt import hp, STATUS_OK, tpe, Trials, fmin

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [9]:
cd '/content/drive/My Drive/Colab Notebooks/dw_matrix/matrix_three/dw_matrix_road_sign'

/content/drive/My Drive/Colab Notebooks/dw_matrix/matrix_three/dw_matrix_road_sign


In [0]:
train = pd.read_pickle('data/train.p')
test = pd.read_pickle('data/test.p')

X_train, y_train = train['features'], train['labels']
X_test, y_test = test['features'], test['labels']

In [0]:
if y_train.ndim == 1: y_train = to_categorical(y_train)
if y_test.ndim == 1: y_test = to_categorical(y_test)

input_shape = X_train.shape[1:]
num_classes = y_train.shape[1]

In [0]:
def train_model(model, X_train, y_train, params_fit={}):
  model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])

  logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
  tensorboard_callbacks = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)
  
  model.fit(
      X_train,
      y_train,
      batch_size=params_fit.get('batch_size', 128),
      epochs=params_fit.get('epochs', 5),
      verbose=params_fit.get('verbose', 1),
      validation_data=params_fit.get('validation_data', (X_train, y_train)),
      callbacks=[tensorboard_callbacks]
  )

  return model


def predict(model_trained, X_test, y_test, scoring=accuracy_score):
  y_test_norm = np.argmax( y_test, axis=1 )   #  valid values

  y_pred_prob = model_trained(X_test)  
  y_pred = np.argmax( y_pred_prob, axis=1 )   #  forecasted values

  return scoring(y_test_norm, y_pred)

In [0]:
def get_cnn_v5(input_shape, num_classes):
  return Sequential([
    Conv2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape=input_shape),
    Conv2D(filters=32, kernel_size=(3, 3), activation='relu', padding='same'),
    MaxPool2D(),
    Dropout(0.3),

    Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same'),
    Conv2D(filters=64, kernel_size=(3, 3), activation='relu'),
    MaxPool2D(),
    Dropout(0.3),

    
    Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same'),
    Conv2D(filters=64, kernel_size=(3, 3), activation='relu'),
    MaxPool2D(),
    Dropout(0.3),

    Flatten(),

    Dense(1024, activation='relu'),  
    Dropout(0.3),

    Dense(1024, activation='relu'),  
    Dropout(0.3),

    Dense(num_classes, activation='softmax'),               
  ])

In [0]:
X_train_gray = color.rgb2gray( X_train ).reshape(-1, 32, 32, 1)
X_test_gray = color.rgb2gray( X_test ).reshape(-1, 32, 32, 1)

In [15]:
model = get_cnn_v5((32, 32, 1), num_classes)
model_trained = train_model(model, X_train_gray, y_train, params_fit={})

predict(model_trained, X_test_gray, y_test)

Epoch 1/5
272/272 [==============================] - 7s 24ms/step - loss: 2.8381 - accuracy: 0.2145 - val_loss: 0.8411 - val_accuracy: 0.7422
Epoch 2/5
272/272 [==============================] - 6s 23ms/step - loss: 0.5477 - accuracy: 0.8244 - val_loss: 0.1382 - val_accuracy: 0.9566
Epoch 3/5
272/272 [==============================] - 6s 22ms/step - loss: 0.2109 - accuracy: 0.9337 - val_loss: 0.0442 - val_accuracy: 0.9869
Epoch 4/5
272/272 [==============================] - 6s 22ms/step - loss: 0.1427 - accuracy: 0.9555 - val_loss: 0.0312 - val_accuracy: 0.9913
Epoch 5/5
272/272 [==============================] - 6s 22ms/step - loss: 0.1117 - accuracy: 0.9651 - val_loss: 0.0234 - val_accuracy: 0.9933


0.9605442176870749

In [0]:
def get_model(params):
  return Sequential([
    Conv2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape=input_shape),
    Conv2D(filters=32, kernel_size=(3, 3), activation='relu', padding='same'),
    MaxPool2D(),
    Dropout(params['dropout_cnn_block_one']),

    Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same'),
    Conv2D(filters=64, kernel_size=(3, 3), activation='relu'),
    MaxPool2D(),
    Dropout(params['dropout_cnn_block_two']),

    
    Conv2D(filters=128, kernel_size=(3, 3), activation='relu', padding='same'),
    Conv2D(filters=128, kernel_size=(3, 3), activation='relu'),
    MaxPool2D(),
    Dropout(params['dropout_cnn_block_three']),

    Flatten(),

    Dense(1024, activation='relu'),  
    Dropout(params['dropout_dense_block_one']),

    Dense(1024, activation='relu'),  
    Dropout(params['dropout_dense_block_two']),

    Dense(num_classes, activation='softmax'),               
  ])

In [0]:
def func_obj(params):
  model = get_model(params)
  model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])

  model.fit(
      X_train,
      y_train,
      batch_size=int(params.get('batch_size', 128)),
      epochs=5,
      verbose=0
  )

  score = model.evaluate(X_test, y_test, verbose=0)
  accuracy = score[1]
  print(params, 'accuracy={}'.format(accuracy))

  return {'loss': -accuracy, 'status': STATUS_OK, 'model': model}

In [18]:
space = {
    'batch_size': hp.quniform('batch_size', 100, 200, 10),
    'dropout_cnn_block_one': hp.uniform('dropout_cnn_block_one', 0.3, 0.5),
    'dropout_cnn_block_two': hp.uniform('dropout_cnn_block_two', 0.3, 0.5),
    'dropout_cnn_block_three': hp.uniform('dropout_cnn_block_three', 0.3, 0.5),

    'dropout_dense_block_one': hp.uniform('dropout_dense_block_one', 0.3, 0.7),
    'dropout_dense_block_two': hp.uniform('dropout_dense_block_two', 0.3, 0.7),
}

best = fmin(
    func_obj,
    space,
    tpe.suggest,
    30,
    Trials()
)

{'batch_size': 130.0, 'dropout_cnn_block_one': 0.39701307807729797, 'dropout_cnn_block_three': 0.37037815132538043, 'dropout_cnn_block_two': 0.3706788615317399, 'dropout_dense_block_one': 0.3490292131455357, 'dropout_dense_block_two': 0.4670155431521912}
accuracy=0.9630385637283325
{'batch_size': 110.0, 'dropout_cnn_block_one': 0.31249557167150016, 'dropout_cnn_block_three': 0.31188616407023567, 'dropout_cnn_block_two': 0.45346565211510387, 'dropout_dense_block_one': 0.6481380472535384, 'dropout_dense_block_two': 0.5469129567584188}
accuracy=0.9510204195976257
{'batch_size': 150.0, 'dropout_cnn_block_one': 0.339803603607721, 'dropout_cnn_block_three': 0.3352991051822026, 'dropout_cnn_block_two': 0.35670721297550684, 'dropout_dense_block_one': 0.6964496606400332, 'dropout_dense_block_two': 0.4691599059805325}
accuracy=0.8804988861083984
{'batch_size': 190.0, 'dropout_cnn_block_one': 0.49177042529660575, 'dropout_cnn_block_three': 0.35662079892325677, 'dropout_cnn_block_two': 0.466252144